In [ ]:
#####設定讀取資料的年份與時段#####

year = 104    #110年或104年
rush_hour = "晨峰"    #晨峰或昏峰

inputLocation = f"{year}{rush_hour}_禁左/"

################################

import pandas as pd

df= pd.read_excel("TRTS-4S屏柵線.xlsx", sheet_name=None)
sheet1 = df["節點編號"]
if rush_hour == '晨峰':
    real_flow = df["晨峰實際流量"]
elif rush_hour == '昏峰':
    real_flow = df["昏峰實際流量"]
print(f'——————————————————————校估{year}年{rush_hour}指派流量——————————————————————')

real_flow = real_flow.set_index("group")


ue_results = pd.read_csv(inputLocation + f"{year}{rush_hour}_UE_results.dat", sep="\t")
ue_results.rename(columns={"tailNode": "A", "headNode": "B"}, inplace=True)

ue_results["link_name"] = ue_results.apply(lambda row: f"{int(row['A'])}_{int(row['B'])}", axis=1)

#look up UE_flow in ue_results to add a new column to sheet1 and sheet2
#for sheet2, we need to find the corresponding link name for A1, B1 and A2, B2
#that is, there are two link to be found for each row in sheet2
#then we need to add up the two link's UE_flow to get the total flow for this row
#so that we can compare with AM flow
sheet1["link_name1"] = sheet1.apply(lambda row: f"{int(row['A1'])}_{int(row['B1'])}", axis=1)
sheet1["link_name2"] = sheet1.apply(lambda row: f"{int(row['A2'])}_{int(row['B2'])}", axis=1)
sheet1 = sheet1.merge(ue_results[["link_name", "UE_flow", "capacity"]], left_on="link_name1", right_on="link_name", how="left")
sheet1 = sheet1.merge(ue_results[["link_name", "UE_flow", "capacity"]], left_on="link_name2", right_on="link_name", how="left", suffixes=('_1', '_2'))
#calculate the total flow and capacity for this row

#reserve the link,AM,UE_flow
sheet1 = sheet1[["group","link", "UE_flow_1", "UE_flow_2"]]

group_names = sheet1['group'].unique()
groups = {i: {'UE_flow_1': 0, 'UE_flow_2': 0} for i in group_names}
for index, row in sheet1.iterrows():
    group = row['group']
    groups[group]['UE_flow_1'] += row['UE_flow_1']
    groups[group]['UE_flow_2'] += row['UE_flow_2']

# UE_flow_1 代表(往南/東)流量， UE_flow_2 代表(往北/西)流量
# {real_flow.loc[group, 'name']}
for group, flows in groups.items():
    print(f"""【{group}】 {real_flow.loc[group, 'name']}
    UE flow 往南/東 = {round(flows['UE_flow_1'])}， diff = {(flows['UE_flow_1'] - real_flow.loc[group, '南/東']) / real_flow.loc[group, '南/東']* 100:.2f}%， CUBE diff = {(flows['UE_flow_1'] - real_flow.loc[group, 'CUBE南/東']) / real_flow.loc[group, 'CUBE南/東']* 100:.2f}%
    UE flow 往北/西 = {round(flows['UE_flow_2'])}， diff = {(flows['UE_flow_2'] - real_flow.loc[group, '北/西']) / real_flow.loc[group, '北/西']* 100:.2f}%， CUBE diff = {(flows['UE_flow_2'] - real_flow.loc[group, 'CUBE北/西']) / real_flow.loc[group, 'CUBE北/西']* 100:.2f}%
    """)

——————————————————————校估104年晨峰指派流量——————————————————————
【SL1】 基隆河
    UE flow 往南/東 = 49133， diff = 15.11%， CUBE diff = 5.91%
    UE flow 往北/西 = 50999， diff = 20.39%， CUBE diff = 20.67%
    
【SL2】 台鐵(市民大道)
    UE flow 往南/東 = 31058， diff = 4.14%， CUBE diff = -8.94%
    UE flow 往北/西 = 37545， diff = -2.43%， CUBE diff = -0.81%
    
【SL3】 國1-市中心
    UE flow 往南/東 = 4779， diff = -22.44%， CUBE diff = -5.71%
    UE flow 往北/西 = 4230， diff = -21.65%， CUBE diff = -9.60%
    
【SL5】 臺北縣市東側運輸走廊
    UE flow 往南/東 = 8365， diff = -7.53%， CUBE diff = 11.64%
    UE flow 往北/西 = 17322， diff = -3.83%， CUBE diff = 19.63%
    
【SL6】 北市北側
    UE flow 往南/東 = 5295， diff = -5.22%， CUBE diff = -6.53%
    UE flow 往北/西 = 3759， diff = -0.49%， CUBE diff = 11.51%
    
【SL7】 臺北縣市西側運輸走廊(淡水河流域)
    UE flow 往南/東 = 41279， diff = 4.07%， CUBE diff = 2.46%
    UE flow 往北/西 = 27519， diff = 16.44%， CUBE diff = 6.45%
    
【SL8】 臺北縣市南側運輸走廊(新店溪&景美溪流域)
    UE flow 往南/東 = 26913， diff = 3.50%， CUBE diff = 1.96%
    UE flow 往北/西 = 33508，

In [4]:
# sheet1 sorted by group and link
sheet1.sort_values(by=['group', 'link'], inplace=True)
pd.set_option('display.max_rows', None)
sheet1[["group", "link", "UE_flow_1", "UE_flow_2"]]
#show all the results in a table

,group,link,UE_flow_1,UE_flow_2
4,SL1,中山橋,1220.382887,723.416713
16,SL1,南湖大橋,3543.943011,2692.536724
5,SL1,大直橋,3146.623240,3034.934129
15,SL1,成功橋,1729.520723,1665.999474
14,SL1,成美橋,1760.931191,543.320901
2,SL1,承德橋,1901.965115,1187.449209
3,SL1,承德橋(機車道),2309.178329,2388.638517
8,SL1,民權大橋,1479.574892,1941.046210
9,SL1,民權大橋(機車道),2270.488624,2479.499783
0,SL1,洲美快速道路,3996.095717,5216.426028
